## Homework 3: Financial Ratio Quantile Strategies - Jingwen Li

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scipy as sp
from datetime import datetime
import quandl
import functools
import seaborn as sns
import nasdaqdatalink as ndl
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from scipy.stats import norm, pearsonr, zscore
from statsmodels.tsa.stattools import adfuller
import warnings
warnings.filterwarnings('ignore')

Steps:
1. Collect historical data for the financial ratios chosen to use in the strategy, such as debt to market cap, return on investment, and price to earnings.
2. Preprocess the data by cleaning it, handling missing values, and normalizing it as needed.
3. Code the algorithm for ranking the securities based on the ratios.
4. Implement the logic for sizing positions by rank.
5. Implement the quantile trading strategy based on the chosen financial ratios and the top-and-bottom decile trading strategy.
6. Divide the data into training and testing sets for backtesting the strategy.
7. Run the backtesting simulation on the training set and evaluate the strategy's performance.
8. Test the strategy on the testing set to check for overfitting.
9. Optimize the strategy by tweaking the parameters and experimenting with different combinations of ratios.
10. Finally, test the strategy on the unseen data and evaluate the performance.

### 1. Collect data

In [7]:
# api =  "key"

# start_date = '2014-09-30'
# end_date = '2022-01-01'

# ndl.export_table('QUOTEMEDIA/PRICES', date = { 'gte': start_date, 'lte': end_date }, ticker = tic, filename='data/QM_PRICES.zip', api_key = api)
# ndl.export_table('ZACKS/MKTV', ticker = tic, filename='data/ZACKS_MKTV.zip', api_key = api)
# ndl.export_table('ZACKS/FC', ticker = tic, filename='data/ZACKS_FC.zip', api_key = api)
# ndl.export_table('ZACKS/FR', ticker = tic, filename='data/ZACKS_FR.zip', api_key = api)
# ndl.export_table('ZACKS/SHRS', ticker = tic, filename='data/ZACKS_SHRS.zip', api_key = api)

### 2. Process Data
    • end-of-day adjusted closing prices are available , over the entire period Jan 2015 through Jan 2022
    • debt/market cap ratio is greater than 0.1 somewhere in the period Jan 2015 through Jan 2022 (preferably more than fleetingly)
    • not in the automotive, financial or insurance sector , over the entire period Jan 2015 through Jan 20225
    • has feasible calculation of the ratios specified below , over the entire period Jan 2015 through Jan 2022, including for at least one PER END DATE no more than one year old. Debt ratio of zero is OK.


In [3]:
zacks = pd.read_csv('ZACKS_MT.csv')
zacks.loc[:,'country_code'].dropna().unique()

array(['US', 'CN', 'HK', 'CA', 'CH', 'GB', 'BR', 'IL', 'BM', 'IE', 'DE',
       'FR', 'NL', 'AT', 'PE', 'SG', 'BE', 'LU', 'AU', 'JP', 'KY', 'TR',
       'CL', 'SE', 'ES', 'IT', 'DK', 'IN', 'ZA', 'MX', 'TW', 'RU', 'CO',
       'TH', 'AR', 'PA', 'JE', 'NZ', 'GR', 'PT', 'KR', 'NO', 'ID', 'FI',
       'HU', 'BS', 'PH'], dtype=object)

In [4]:
zacks.loc[:,'zacks_x_sector_desc'].dropna().unique()

array(['Computer and Technology', 'Industrial Products', 'Unclassified',
       'Consumer Discretionary', 'Medical', 'Finance', 'Aerospace',
       'Transportation', 'Construction', 'Retail/Wholesale',
       'Business Services', 'Basic Materials', 'Oils/Energy',
       'Consumer Staples', 'Auto/Tires/Trucks', 'Utilities',
       'Multi-Sector Conglomerates', 'Index'], dtype=object)

2.1 Filter tickers

    • Filter non-US companies
    • Filter Finance, Autp, Multi, Unclassified and Business Service Sector
    • only consider stocks listed on NYSE and NASDAQ
    • Only consider stocks having active flag

993 stocks left

In [6]:
zacks_filtered = zacks.loc[(zacks['country_code'] == 'US') 
& (zacks['zacks_x_sector_desc'].isin(['Finance', 'Auto/Tires/Trucks','Multi-Sector Conglomerates', 'Unclassified', 'Business Services']) == False) 
& (zacks['exchange'].isin(['NYSE', 'NSDAQ']) )
& (zacks['active_ticker_flag'] == 'Y') 
& (zacks['asset_type'] == 'COM')].iloc[:,1]
tic = zacks_filtered.tolist()
print(f'# of stcoks left after filteration: {len(tic)}')

# of stcoks left after filteration: 993


2.2 loading PRICES
 
    • Filtered stocks not having closing prices available over the entire period Jan 2015 through Jan 2022


In [8]:
adj_price = pd.read_csv('data/QM_PRICES.csv', usecols=['ticker', 'date', 'adj_close'])
adj_price.sort_values(by=['ticker', 'date'], inplace=True)
adj_price

,ticker,date,adj_close
0,A,2014-09-30,38.386511
1,A,2014-10-01,37.861038
2,A,2014-10-02,37.611775
3,A,2014-10-03,38.339353
4,A,2014-10-06,38.110301
...,...,...,...
1450377,ZWS,2021-12-27,36.207903
1450378,ZWS,2021-12-28,36.396537
1450379,ZWS,2021-12-29,36.466034
1450380,ZWS,2021-12-30,35.979556


In [9]:
adj_price_count = adj_price.groupby('ticker')['adj_close'].count()
adj_price_filtered = adj_price[adj_price.groupby('ticker')['adj_close'].transform(lambda x: x.count()) > 1800]['ticker']
print(f'# of stcoks left after filteration: {adj_price_filtered.nunique()}')

# of stcoks left after filteration: 672


2.3 loading FR

    • Filter stocks with debt/market cap ratio that is not constantly greater than 0.1 in the period Jan 2015 through Jan 2022

In [10]:
fr = pd.read_csv('data/ZACKS_FR.csv', usecols=['per_end_date', 'ticker', 'tot_debt_tot_equity', 'ret_invst', 'per_type'])
mask = (fr['per_end_date'] >= '2014-09-30') & (fr['per_end_date'] <= '2022-01-01') & fr['ticker'].isin(adj_price_filtered) & (fr['per_type'] == 'Q')
fr = fr[mask]
fr.sort_values(by=['ticker', 'per_end_date'], inplace=True)
fr

,ticker,per_end_date,per_type,tot_debt_tot_equity,ret_invst
32,A,2014-10-31,Q,0.3135,0.2297
31,A,2015-01-31,Q,0.3910,1.5768
30,A,2015-04-30,Q,0.3980,1.5816
29,A,2015-07-31,Q,0.4031,1.9615
28,A,2015-10-31,Q,0.3969,2.4034
...,...,...,...,...,...
63891,ZWS,2020-12-31,Q,NaN,NaN
63890,ZWS,2021-03-31,Q,0.7983,0.3728
63889,ZWS,2021-06-30,Q,0.7508,0.7418
63888,ZWS,2021-09-30,Q,0.7242,0.5679


In [11]:
fr[fr['tot_debt_tot_equity'] > 0.1].groupby('ticker')['tot_debt_tot_equity'].count()
fr_filtered = fr[fr.groupby('ticker')['tot_debt_tot_equity'].transform(lambda x: x.count()) > 20]['ticker']
print(f'# of stcoks left after filteration: {fr_filtered.nunique()}')

# of stcoks left after filteration: 607


2.4 loading MKTV

    • Filter stocks having missing maket value data over the entire period Jan 2015 through Jan 2022


In [12]:
mkt_val = pd.read_csv('data/ZACKS_MKTV.csv', usecols=['per_end_date', 'ticker', 'mkt_val', 'per_type'])
mask = (mkt_val['per_end_date'] >= '2014-09-30') & (mkt_val['per_end_date'] <= '2022-01-01') & mkt_val['ticker'].isin(fr_filtered) & (mkt_val['per_type'] == 'Q')
mkt_val = mkt_val[mask]
mkt_val.sort_values(by=['ticker', 'per_end_date'], inplace=True)
mkt_val

,ticker,per_type,per_end_date,mkt_val
34,A,Q,2014-09-30,19003.57
35,A,Q,2014-12-31,13751.83
36,A,Q,2015-03-31,13952.91
37,A,Q,2015-06-30,12854.55
38,A,Q,2015-09-30,11377.07
...,...,...,...,...
67242,ZWS,Q,2020-12-31,4749.85
67243,ZWS,Q,2021-03-31,5637.67
67244,ZWS,Q,2021-06-30,6060.11
67245,ZWS,Q,2021-09-30,7801.56


In [13]:
mkt_val_count = mkt_val.groupby('ticker')['mkt_val'].count()
mkt_val_filtered = mkt_val[mkt_val.groupby('ticker')['mkt_val'].transform(lambda x: x.count()) >= 30]['ticker']
print(f'# of stcoks left after filteration: {mkt_val_filtered.nunique()}')

# of stcoks left after filteration: 598


2.5 loading SHRS

    • Filter stocks missing outstanding shares data


In [14]:
shrs = pd.read_csv('data/ZACKS_SHRS.csv', usecols=['per_end_date', 'ticker', 'shares_out', 'per_type'])
mask = (shrs['per_end_date'] >= '2014-09-30') & (shrs['per_end_date'] <= '2022-01-01') & shrs['ticker'].isin(mkt_val_filtered) & (shrs['per_type'] == 'Q')
shrs = shrs[mask]
shrs.sort_values(by=['ticker', 'per_end_date'], inplace=True)
shrs

,ticker,per_type,per_end_date,shares_out
35,A,Q,2014-10-31,333.51
36,A,Q,2015-01-31,336.00
37,A,Q,2015-04-30,335.81
38,A,Q,2015-07-31,333.19
39,A,Q,2015-10-31,331.40
...,...,...,...,...
67242,ZWS,Q,2020-12-31,120.28
67243,ZWS,Q,2021-03-31,119.72
67244,ZWS,Q,2021-06-30,121.11
67245,ZWS,Q,2021-09-30,121.35


In [15]:
shrs.groupby('ticker')['shares_out'].count().unique()
shrs_filtered = shrs[shrs.groupby('ticker')['shares_out'].transform(lambda x: x.count()) > 28]['ticker']
print(f'# of stcoks left after filteration: {shrs_filtered.nunique()}')

# of stcoks left after filteration: 598


2.6 loading FC

    • Filter stocks missing filing date
    • Filter stocks missing both total long term debt and net long term debt
    • Filter stocks missing both net basic eps and net diluted eps

In [16]:
fc = pd.read_csv('data/ZACKS_FC.csv', usecols=['per_end_date', 'filing_date', 'ticker', 'net_lterm_debt', 'tot_lterm_debt', 'eps_diluted_net', 'eps_basic_net', 'per_type'])
mask = (fc['per_end_date'] >= '2014-09-30') & (fc['per_end_date'] <= '2022-01-01') & fc['ticker'].isin(shrs_filtered) & (fc['per_type'] == 'Q')
fc = fc[mask]
fc.sort_values(by=['ticker', 'per_end_date'], inplace=True)
fc

,ticker,per_end_date,per_type,filing_date,tot_lterm_debt,net_lterm_debt,eps_basic_net,eps_diluted_net
5,A,2014-10-31,Q,2014-12-22,1663.0,99.0,0.05,0.05
67,A,2015-01-31,Q,2015-03-10,1658.0,NaN,0.19,0.19
66,A,2015-04-30,Q,2015-06-05,1656.0,NaN,0.26,0.26
65,A,2015-07-31,Q,2015-09-02,1655.0,NaN,0.33,0.33
64,A,2015-10-31,Q,2015-12-21,1655.0,NaN,0.42,0.42
...,...,...,...,...,...,...,...,...
63891,ZWS,2020-12-31,Q,NaN,1118.0,NaN,NaN,NaN
63890,ZWS,2021-03-31,Q,2021-04-27,1189.3,-0.5,0.42,0.40
63889,ZWS,2021-06-30,Q,2021-07-20,1189.5,-1.1,0.61,0.59
63888,ZWS,2021-09-30,Q,2021-10-26,1189.3,-1.7,0.53,0.51


In [17]:
fc = fc.dropna(subset=['tot_lterm_debt','net_lterm_debt'], how = 'all')
fc = fc.dropna(subset=['eps_basic_net', 'eps_diluted_net'], how = 'all')
fc['net_lterm_debt'].fillna(fc['tot_lterm_debt'], inplace=True)
fc['eps_diluted_net'].fillna(fc['eps_basic_net'], inplace=True)
fc = fc.drop(columns = ['tot_lterm_debt', 'eps_basic_net'])
fc_filtered = fc[fc.groupby('ticker')['filing_date'].transform(lambda x: x.count()) >= 28]['ticker']
print(f'# of stcoks left after filteration: {fc_filtered.nunique()}')

# of stcoks left after filteration: 560


2.7 Merge table

    a. combine financial data

In [18]:
fin_data = pd.merge(pd.merge(pd.merge(fc, shrs, on=['ticker', 'per_end_date', 'per_type'], how='inner'),
mkt_val, on=['ticker', 'per_end_date', 'per_type'], how='inner'),
fr, on=['ticker', 'per_end_date', 'per_type'], how='inner')
financial = fin_data[fin_data.groupby('ticker')['per_end_date'].transform(lambda x: x.count()) >= 28]
financial_filtered = financial['ticker']
print(f'# of stcoks left after filteration: {financial_filtered.nunique()}')

# of stcoks left after filteration: 482


    b. Combine financial and price

In [19]:
mask = adj_price['ticker'].isin(financial['ticker'].unique())
price = adj_price[mask]

In [20]:
data = pd.merge(financial, price, left_on=['ticker', 'filing_date'], right_on=['ticker', 'date'], how='outer')
data['date'].fillna(data['filing_date'], inplace=True)
data.sort_values(by=['ticker', 'date'], inplace=True)
data = data.drop(columns = 'per_type')
data

,ticker,per_end_date,filing_date,net_lterm_debt,eps_diluted_net,shares_out,mkt_val,tot_debt_tot_equity,ret_invst,date,adj_close
14446,AAP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-09-30,121.765847
14447,AAP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-10-01,121.971438
14448,AAP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-10-02,124.260972
14449,AAP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-10-03,126.279500
14450,AAP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-10-06,125.616003
...,...,...,...,...,...,...,...,...,...,...,...
881616,ZTS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-12-28,241.902749
881617,ZTS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-12-29,244.656033
881618,ZTS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-12-30,242.804004
881619,ZTS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-12-31,241.684864


In [21]:
cal = pd.merge(financial, price, left_on=['ticker', 'per_end_date'], right_on=['ticker', 'date'], how='outer')
cal['date'].fillna(cal['per_end_date'], inplace=True)
cal.sort_values(by=['ticker', 'date'], inplace=True)
cal = cal[['date','filing_date','ticker','adj_close','net_lterm_debt','eps_diluted_net','shares_out','mkt_val','tot_debt_tot_equity','ret_invst']]
cal['tot_debt_tot_share'] = cal['tot_debt_tot_equity'] * cal['adj_close']
cal['mkt_val_div_price'] = cal['mkt_val'] / cal['adj_close']
cal = cal[['filing_date','ticker','tot_debt_tot_share', 'mkt_val_div_price']]
cal.dropna(subset = 'filing_date', inplace = True)
cal

,filing_date,ticker,tot_debt_tot_share,mkt_val_div_price
0,2014-11-12,AAP,109.759734,78.089548
1,2015-03-03,AAP,121.698564,78.080807
2,2015-06-02,AAP,105.023713,78.222622
3,2015-08-25,AAP,94.633221,78.216615
4,2015-11-17,AAP,95.093545,78.232053
...,...,...,...,...
14441,2021-02-16,ZTS,311.042574,482.497334
14442,2021-05-06,ZTS,273.012246,481.470427
14443,2021-08-05,ZTS,304.446283,479.704103
14444,2021-11-04,ZTS,270.442767,478.278281


In [22]:
df_sec = pd.merge(data, cal, on=['ticker', 'filing_date'],how='outer')
df_sec.sort_values(by=['ticker', 'date'], inplace=True)
df_sec['date'] = pd.to_datetime(df_sec['date'])
# df_sec.set_index('date', inplace=True)
df_sec

,ticker,per_end_date,filing_date,net_lterm_debt,eps_diluted_net,shares_out,mkt_val,tot_debt_tot_equity,ret_invst,date,adj_close,tot_debt_tot_share,mkt_val_div_price
0,AAP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-09-30,121.765847,NaN,NaN
1,AAP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-10-01,121.971438,NaN,NaN
2,AAP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-10-02,124.260972,NaN,NaN
3,AAP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-10-03,126.279500,NaN,NaN
4,AAP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-10-06,125.616003,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
881602,ZTS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-12-28,241.902749,NaN,NaN
881603,ZTS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-12-29,244.656033,NaN,NaN
881604,ZTS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-12-30,242.804004,NaN,NaN
881605,ZTS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-12-31,241.684864,NaN,NaN


    c. forward filling NaN between two filing dates

In [23]:
group1 = df_sec.groupby('ticker', group_keys = False)
# Forward fill the 'filing_date' column within each group
df_sec['filing_date'] = group1['filing_date'].apply(lambda x: x.ffill())

df_sec.dropna(subset=['filing_date'], inplace=True)

cols_to_fill = ['net_lterm_debt','eps_diluted_net','shares_out','mkt_val','tot_debt_tot_equity','ret_invst', 'tot_debt_tot_share','mkt_val_div_price']
group = df_sec.groupby(['ticker', 'filing_date'], group_keys= False)
df_sec[cols_to_fill] = group[cols_to_fill].apply(lambda x: x.ffill())
df_sec.sort_values(by=['ticker', 'date'], inplace=True)
df_sec

,ticker,per_end_date,filing_date,net_lterm_debt,eps_diluted_net,shares_out,mkt_val,tot_debt_tot_equity,ret_invst,date,adj_close,tot_debt_tot_share,mkt_val_div_price
1799,AAP,2014-09-30,2014-11-12,1730.15,1.66,72.97,9508.64,0.9014,3.3421,2014-11-12,136.689873,109.759734,78.089548
31,AAP,NaN,2014-11-12,1730.15,1.66,72.97,9508.64,0.9014,3.3421,2014-11-13,136.437557,109.759734,78.089548
32,AAP,NaN,2014-11-12,1730.15,1.66,72.97,9508.64,0.9014,3.3421,2014-11-14,135.325497,109.759734,78.089548
33,AAP,NaN,2014-11-12,1730.15,1.66,72.97,9508.64,0.9014,3.3421,2014-11-17,136.278691,109.759734,78.089548
34,AAP,NaN,2014-11-12,1730.15,1.66,72.97,9508.64,0.9014,3.3421,2014-11-18,136.661838,109.759734,78.089548
...,...,...,...,...,...,...,...,...,...,...,...,...,...
881602,ZTS,NaN,2021-11-04,-600.00,1.16,473.13,91852.65,1.4082,4.8967,2021-12-28,241.902749,270.442767,478.278281
881603,ZTS,NaN,2021-11-04,-600.00,1.16,473.13,91852.65,1.4082,4.8967,2021-12-29,244.656033,270.442767,478.278281
881604,ZTS,NaN,2021-11-04,-600.00,1.16,473.13,91852.65,1.4082,4.8967,2021-12-30,242.804004,270.442767,478.278281
881605,ZTS,NaN,2021-11-04,-600.00,1.16,473.13,91852.65,1.4082,4.8967,2021-12-31,241.684864,270.442767,478.278281


2.8 Calculate Financial Ratios

In [24]:
df_sec['debt_to_mkt_cap'] = df_sec['tot_debt_tot_share'] / df_sec['adj_close']
df_sec['ret_on_inv'] = df_sec['ret_invst']* (df_sec['net_lterm_debt'] + df_sec['mkt_val']) / (df_sec['net_lterm_debt'] + df_sec['mkt_val_div_price'] * df_sec['adj_close'])
df_sec['price_to_earnings'] = df_sec['adj_close'] / df_sec['eps_diluted_net']
df_sec.set_index('date',inplace = True)


In [25]:
def_sec = df_sec.replace([np.inf, -np.inf], np.nan)
df_sec

,ticker,per_end_date,filing_date,net_lterm_debt,eps_diluted_net,shares_out,mkt_val,tot_debt_tot_equity,ret_invst,adj_close,tot_debt_tot_share,mkt_val_div_price,debt_to_mkt_cap,ret_on_inv,price_to_earnings
date,,,,,,,,,,,,,,,
2014-11-12,AAP,2014-09-30,2014-11-12,1730.15,1.66,72.97,9508.64,0.9014,3.3421,136.689873,109.759734,78.089548,0.802984,3.028100,82.343297
2014-11-13,AAP,NaN,2014-11-12,1730.15,1.66,72.97,9508.64,0.9014,3.3421,136.437557,109.759734,78.089548,0.804469,3.032918,82.191299
2014-11-14,AAP,NaN,2014-11-12,1730.15,1.66,72.97,9508.64,0.9014,3.3421,135.325497,109.759734,78.089548,0.811079,3.054335,81.521384
2014-11-17,AAP,NaN,2014-11-12,1730.15,1.66,72.97,9508.64,0.9014,3.3421,136.278691,109.759734,78.089548,0.805406,3.035959,82.095597
2014-11-18,AAP,NaN,2014-11-12,1730.15,1.66,72.97,9508.64,0.9014,3.3421,136.661838,109.759734,78.089548,0.803148,3.028635,82.326408
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-28,ZTS,NaN,2021-11-04,-600.00,1.16,473.13,91852.65,1.4082,4.8967,241.902749,270.442767,478.278281,1.117981,3.882269,208.536853
2021-12-29,ZTS,NaN,2021-11-04,-600.00,1.16,473.13,91852.65,1.4082,4.8967,244.656033,270.442767,478.278281,1.105400,3.838354,210.910374
2021-12-30,ZTS,NaN,2021-11-04,-600.00,1.16,473.13,91852.65,1.4082,4.8967,242.804004,270.442767,478.278281,1.113832,3.867784,209.313797


    a. debt to market cap ratio table

In [26]:
debt_to_mkt_cap = df_sec.pivot_table(values='debt_to_mkt_cap', columns='ticker', index=df_sec.index)
debt_to_mkt_cap = debt_to_mkt_cap.replace([np.inf, -np.inf], np.nan)
dmc=debt_to_mkt_cap['2015-01':]
# dmc=dmc.drop(dmc.columns[-2:], axis=1)
dmc.round(2)

ticker,AAP,ABBV,ABC,ABG,ABT,ACCO,ACM,ADM,AEE,AES,...,WWE,WWW,WY,X,XOM,XPO,XYL,YUM,ZBH,ZTS
date,,,,,,,,,,,,,,,,,,,,,
2015-01-02,0.74,2.77,0.87,1.88,0.33,0.97,0.49,0.27,0.80,2.87,...,0.14,0.98,0.81,1.31,0.12,0.34,0.53,1.30,0.23,2.24
2015-01-05,0.75,2.82,0.88,1.93,0.33,0.99,0.51,0.28,0.81,2.96,...,0.15,0.99,0.81,1.38,0.12,0.35,0.56,1.33,0.22,2.25
2015-01-06,0.75,2.84,0.87,1.97,0.34,1.01,0.52,0.28,0.82,3.02,...,0.15,1.01,0.81,1.42,0.12,0.36,0.56,1.35,0.22,2.27
2015-01-07,0.74,2.73,0.85,1.91,0.33,1.02,0.51,0.28,0.81,3.02,...,0.15,1.00,0.81,1.42,0.12,0.36,0.56,1.30,0.22,2.23
2015-01-08,0.73,2.70,0.85,1.89,0.33,0.98,0.49,0.28,0.81,2.97,...,0.15,0.98,0.80,1.39,0.12,0.36,0.55,1.28,0.21,2.19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-27,0.28,4.72,10.32,1.36,0.44,1.49,0.65,0.37,1.22,3.33,...,1.86,1.64,0.45,0.50,0.32,3.65,0.83,-1.26,0.67,1.11
2021-12-28,0.28,4.72,10.28,1.35,0.44,1.48,0.64,0.36,1.21,3.32,...,1.85,1.65,0.44,0.51,0.32,3.65,0.82,-1.26,0.67,1.12
2021-12-29,0.28,4.68,10.21,1.35,0.44,1.46,0.64,0.36,1.20,3.31,...,1.83,1.62,0.43,0.51,0.32,3.62,0.82,-1.26,0.67,1.11


    b.return on investment ratio table

In [27]:
ret_on_inv = df_sec.pivot_table(values='ret_on_inv', columns='ticker', index=df_sec.index)
ret_on_inv = ret_on_inv.replace([np.inf, -np.inf], np.nan)
roi = ret_on_inv['2015-01':]
# roi = roi.drop(roi.columns[-2:], axis=1)
roi.round(2)

ticker,AAP,ABBV,ABC,ABG,ABT,ACCO,ACM,ADM,AEE,AES,...,WWE,WWW,WY,X,XOM,XPO,XYL,YUM,ZBH,ZTS
date,,,,,,,,,,,,,,,,,,,,,
2015-01-02,2.82,2.34,1.42,2.55,1.52,1.64,2.25,2.85,1.91,1.99,...,-2.84,2.47,1.55,-4.41,4.22,-0.48,2.93,6.77,1.93,2.90
2015-01-05,2.86,2.38,1.43,2.61,1.52,1.68,2.36,2.96,1.93,2.05,...,-2.90,2.50,1.55,-4.65,4.33,-0.50,3.10,6.91,1.86,2.91
2015-01-06,2.86,2.39,1.42,2.64,1.53,1.72,2.39,3.02,1.96,2.10,...,-2.89,2.56,1.55,-4.81,4.36,-0.51,3.11,7.00,1.88,2.94
2015-01-07,2.81,2.31,1.40,2.58,1.52,1.74,2.34,2.98,1.94,2.09,...,-3.02,2.53,1.55,-4.79,4.31,-0.51,3.09,6.78,1.83,2.89
2015-01-08,2.78,2.29,1.39,2.57,1.49,1.66,2.26,3.02,1.94,2.06,...,-3.05,2.47,1.53,-4.68,4.24,-0.51,3.07,6.66,1.82,2.85
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-27,3.56,2.86,5.70,6.28,3.35,1.13,1.72,1.61,1.77,1.91,...,6.57,-0.06,2.74,14.58,3.08,0.49,2.14,14.00,0.97,3.85
2021-12-28,3.54,2.86,5.68,6.23,3.38,1.12,1.72,1.59,1.75,1.91,...,6.53,-0.06,2.73,14.84,3.09,0.49,2.12,14.01,0.97,3.88
2021-12-29,3.50,2.84,5.64,6.24,3.36,1.11,1.71,1.58,1.74,1.90,...,6.46,-0.06,2.67,14.95,3.12,0.49,2.12,13.94,0.97,3.84


    c. price to earnings ratio table

In [28]:
price_to_earnings = df_sec.pivot_table(values='price_to_earnings', columns='ticker', index=df_sec.index)
price_to_earnings = price_to_earnings.replace([np.inf, -np.inf], np.nan)
pe = price_to_earnings['2015-01':]
# pe = pe.drop(pe.columns[-2:], axis=1)
pe.round(2)

ticker,AAP,ABBV,ABC,ABG,ABT,ACCO,ACM,ADM,AEE,AES,...,WWE,WWW,WY,X,XOM,XPO,XYL,YUM,ZBH,ZTS
date,,,,,,,,,,,,,,,,,,,,,
2015-01-02,89.30,149.97,274.65,70.45,106.78,25.87,46.93,36.41,30.51,15.55,...,-133.61,45.82,12.39,-17.48,33.94,-104.47,59.26,50.14,104.65,124.12
2015-01-05,88.12,147.15,272.31,68.52,106.80,25.37,44.87,35.15,30.21,15.11,...,-131.16,45.38,12.39,-16.67,33.02,-101.23,55.57,49.12,108.55,123.38
2015-01-06,88.06,146.42,273.80,67.31,105.59,24.73,44.42,34.46,29.86,14.77,...,-131.50,44.29,12.37,-16.16,32.84,-97.66,55.24,48.52,107.63,122.18
2015-01-07,89.95,152.34,279.27,69.45,106.45,24.52,45.34,34.97,30.09,14.79,...,-126.05,44.75,12.41,-16.20,33.17,-98.07,55.68,50.12,110.31,124.70
2015-01-08,90.74,153.93,279.91,69.91,108.63,25.58,46.74,34.46,30.19,15.02,...,-124.61,45.90,12.54,-16.55,33.73,-98.61,56.10,51.00,111.48,126.62
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-27,85.17,71.96,62.63,22.21,118.38,37.04,116.30,69.73,51.69,47.97,...,92.95,NaN,58.57,3.37,37.93,-156.35,185.34,77.38,183.71,210.47
2021-12-28,85.75,71.95,62.85,22.48,117.55,37.36,116.82,70.64,52.21,48.11,...,93.48,NaN,58.87,3.33,37.80,-156.27,187.27,77.36,184.29,208.54
2021-12-29,86.80,72.47,63.31,22.41,118.15,37.82,117.27,70.73,52.50,48.19,...,94.51,NaN,60.08,3.31,37.47,-157.59,187.02,77.74,184.32,210.91


2.9 Thoughts on meaningful combinations of 3 calculated financial ratios for quantile trading

One combination that is meaningful to use is: ROI * (1- D/MC)

    This metric combines the debt to market cap ratio with the return on investment ratio to give a measure of the return on investment after accounting for the level of debt. This is important because companies with high levels of debt may have a lower return on investment due to the interest expenses associated with servicing that debt. By adjusting the return on investment ratio for the level of debt, we can have a more accurate measure of a company's investment performance and potential.

In [29]:
df_sec['adj_roi'] = df_sec['ret_on_inv'] * (1-df_sec['debt_to_mkt_cap'])

adj_roi = df_sec.pivot_table(values='adj_roi', columns='ticker', index=df_sec.index)
adj_roi= adj_roi.replace([np.inf, -np.inf], np.nan)
adj_roi = adj_roi['2015-01':]
# adj_roi = adj_roi.drop(adj_roi.columns[-2:], axis=1)
adj_roi.round(2)

ticker,AAP,ABBV,ABC,ABG,ABT,ACCO,ACM,ADM,AEE,AES,...,WWE,WWW,WY,X,XOM,XPO,XYL,YUM,ZBH,ZTS
date,,,,,,,,,,,,,,,,,,,,,
2015-01-02,0.73,-4.15,0.19,-2.24,1.01,0.05,1.15,2.09,0.38,-3.73,...,-2.44,0.05,0.30,1.39,3.72,-0.32,1.39,-2.06,1.49,-3.59
2015-01-05,0.71,-4.35,0.18,-2.43,1.01,0.02,1.15,2.14,0.37,-4.01,...,-2.48,0.03,0.30,1.76,3.81,-0.32,1.36,-2.29,1.45,-3.65
2015-01-06,0.71,-4.40,0.18,-2.56,1.02,-0.02,1.15,2.17,0.36,-4.24,...,-2.47,-0.03,0.30,2.03,3.83,-0.33,1.36,-2.43,1.46,-3.74
2015-01-07,0.74,-3.99,0.20,-2.34,1.01,-0.04,1.15,2.15,0.37,-4.23,...,-2.56,-0.00,0.30,2.01,3.80,-0.32,1.37,-2.06,1.44,-3.55
2015-01-08,0.76,-3.89,0.21,-2.29,1.00,0.04,1.15,2.17,0.37,-4.07,...,-2.58,0.06,0.31,1.82,3.74,-0.32,1.37,-1.88,1.43,-3.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-27,2.55,-10.64,-53.12,-2.28,1.87,-0.56,0.61,1.02,-0.39,-4.46,...,-5.67,0.04,1.52,7.28,2.11,-1.30,0.37,31.71,0.32,-0.41
2021-12-28,2.54,-10.64,-52.74,-2.16,1.87,-0.54,0.61,1.01,-0.37,-4.42,...,-5.57,0.04,1.52,7.32,2.11,-1.30,0.39,31.72,0.32,-0.46
2021-12-29,2.52,-10.47,-51.96,-2.19,1.87,-0.51,0.61,1.01,-0.36,-4.41,...,-5.37,0.04,1.51,7.33,2.12,-1.27,0.39,31.48,0.32,-0.40


Another combination could also be used is: 0.5 P/E + 0.5 ROI

    The P/E ratio and ROI can provide a more comprehensive view of a company's performance. A high P/E ratio and a high ROI suggest that a company is profitable and growing, and that its stock is likely overvalued. A low P/E ratio and a low ROI suggest that a company is not performing well and that its stock is likely undervalued.



In [30]:
df_sec['pe_roi'] = 0.5 * df_sec['ret_on_inv'] + 0.5 * df_sec['price_to_earnings']

pe_roi = df_sec.pivot_table(values='pe_roi', columns='ticker', index=df_sec.index)
pe_roi = pe_roi.replace([np.inf, -np.inf], np.nan)
pe_roi = pe_roi['2015-01':]
# pe_roi = pe_roi.drop(pe_roi.columns[-2:], axis=1)
pe_roi.round(2)

ticker,AAP,ABBV,ABC,ABG,ABT,ACCO,ACM,ADM,AEE,AES,...,WWE,WWW,WY,X,XOM,XPO,XYL,YUM,ZBH,ZTS
date,,,,,,,,,,,,,,,,,,,,,
2015-01-02,46.06,76.16,138.04,36.50,54.15,13.76,24.59,19.63,16.21,8.77,...,-68.23,24.15,6.97,-10.95,19.08,-52.48,31.10,28.46,53.29,63.51
2015-01-05,45.49,74.77,136.87,35.56,54.16,13.52,23.62,19.05,16.07,8.58,...,-67.03,23.94,6.97,-10.66,18.67,-50.86,29.33,28.02,55.21,63.15
2015-01-06,45.46,74.41,137.61,34.97,53.56,13.22,23.40,18.74,15.91,8.43,...,-67.19,23.43,6.96,-10.48,18.60,-49.08,29.18,27.76,54.76,62.56
2015-01-07,46.38,77.32,140.33,36.02,53.98,13.13,23.84,18.97,16.01,8.44,...,-64.54,23.64,6.98,-10.50,18.74,-49.29,29.39,28.45,56.07,63.79
2015-01-08,46.76,78.11,140.65,36.24,55.06,13.62,24.50,18.74,16.06,8.54,...,-63.83,24.19,7.04,-10.62,18.98,-49.56,29.59,28.83,56.65,64.73
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-27,44.36,37.41,34.16,14.25,60.87,19.09,59.01,35.67,26.73,24.94,...,49.76,NaN,30.66,8.98,20.50,-77.93,93.74,45.69,92.34,107.16
2021-12-28,44.65,37.41,34.26,14.36,60.46,19.24,59.27,36.11,26.98,25.01,...,50.00,NaN,30.80,9.09,20.45,-77.89,94.70,45.68,92.63,106.21
2021-12-29,45.15,37.66,34.48,14.33,60.76,19.46,59.49,36.16,27.12,25.05,...,50.48,NaN,31.37,9.13,20.30,-78.55,94.57,45.84,92.64,107.37


2.10 price table

In [40]:
adj_close = df_sec.pivot_table(values='adj_close', columns='ticker', index=df_sec.index)
adj_close = adj_close['2015-01':]
# adj_close = adj_close.drop(adj_close.columns[-2:], axis=1)
adj_close.round(2)

ticker,AAP,ABBV,ABC,ABG,ABT,ACCO,ACM,ADM,AEE,AES,...,WWE,WWW,WY,X,XOM,XPO,XYL,YUM,ZBH,ZTS
date,,,,,,,,,,,,,,,,,,,,,
2015-01-02,148.23,46.49,79.65,76.09,38.44,7.50,30.03,41.51,36.61,10.42,...,10.69,26.12,26.63,24.82,64.16,24.03,34.37,44.62,105.69,40.96
2015-01-05,146.28,45.62,78.97,74.00,38.45,7.36,28.72,40.07,36.25,10.12,...,10.49,25.87,26.63,23.66,62.40,23.28,32.23,43.72,109.64,40.72
2015-01-06,146.17,45.39,79.40,72.69,38.01,7.17,28.43,39.28,35.83,9.89,...,10.52,25.25,26.60,22.95,62.07,22.46,32.04,43.18,108.71,40.32
2015-01-07,149.31,47.22,80.99,75.01,38.32,7.11,29.01,39.87,36.10,9.91,...,10.08,25.51,26.68,23.00,62.70,22.56,32.29,44.61,111.41,41.15
2015-01-08,150.62,47.72,81.17,75.50,39.11,7.42,29.92,39.29,36.23,10.06,...,9.97,26.16,26.97,23.51,63.74,22.68,32.54,45.39,112.59,41.78
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-27,228.24,128.09,130.27,167.49,138.50,7.78,75.59,64.85,85.30,23.02,...,48.33,28.25,37.48,23.51,59.55,76.61,116.77,135.41,126.76,244.14
2021-12-28,229.82,128.07,130.72,169.52,137.53,7.85,75.93,65.70,86.14,23.09,...,48.61,28.01,37.67,23.21,59.35,76.57,117.98,135.38,127.16,241.90
2021-12-29,232.62,129.00,131.69,169.00,138.24,7.94,76.23,65.78,86.62,23.13,...,49.15,28.58,38.45,23.09,58.83,77.22,117.82,136.05,127.18,244.66


### 2. Position table

    a. creating z-score tables for each ratio

In [111]:
z_pe = pe.apply(zscore)
z_pe

ticker,AAP,ABBV,ABC,ABG,ABT,ACCO,ACM,ADM,AEE,AES,...,WWE,WWW,WY,X,XOM,XPO,XYL,YUM,ZBH,ZTS
date,,,,,,,,,,,,,,,,,,,,,
2015-01-02,-0.194718,0.168124,1.513807,1.061798,-0.280048,-0.001420,-0.211728,-1.148039,-0.648877,-0.217513,...,-0.705013,NaN,-0.125490,-0.510651,-0.034434,-1.659233,-0.920789,-0.434350,-0.022901,-0.133502
2015-01-05,-0.221301,0.160767,1.495474,0.986914,-0.279824,-0.004546,-0.212314,-1.203629,-0.652126,-0.219888,...,-0.700777,NaN,-0.125490,-0.499618,-0.043352,-1.624629,-0.953699,-0.447941,-0.007009,-0.136855
2015-01-06,-0.222700,0.158868,1.507140,0.939977,-0.291253,-0.008591,-0.212442,-1.234203,-0.655870,-0.221715,...,-0.701354,NaN,-0.125524,-0.492767,-0.045037,-1.586454,-0.956615,-0.455984,-0.010755,-0.142271
2015-01-07,-0.179963,0.174295,1.549996,1.023102,-0.283185,-0.009878,-0.212182,-1.211349,-0.653398,-0.221594,...,-0.691919,NaN,-0.125430,-0.493301,-0.041842,-1.590849,-0.952727,-0.434535,0.000143,-0.130923
2015-01-08,-0.162156,0.178448,1.554996,1.040659,-0.262568,-0.003259,-0.211781,-1.233894,-0.652268,-0.220375,...,-0.689416,NaN,-0.125097,-0.498106,-0.036540,-1.596616,-0.948978,-0.422886,0.004910,-0.122284
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-27,-0.287984,-0.035234,-0.148808,-0.804912,-0.170741,0.068342,-0.191932,0.322446,-0.423376,-0.043574,...,-0.312230,NaN,-0.010370,-0.228430,0.003805,-2.213287,0.204388,-0.070932,0.298968,0.254975
2021-12-28,-0.274726,-0.035262,-0.147095,-0.794493,-0.178548,0.070334,-0.191784,0.362498,-0.417908,-0.042813,...,-0.311303,NaN,-0.009633,-0.229007,0.002629,-2.212416,0.221586,-0.071157,0.301309,0.246293
2021-12-29,-0.251140,-0.033908,-0.143446,-0.797162,-0.172870,0.073180,-0.191654,0.366224,-0.414827,-0.042378,...,-0.309516,NaN,-0.006615,-0.229238,-0.000549,-2.226584,0.219349,-0.066070,0.301426,0.256972
